### Fine-Tune Command

In [1]:
!pip install transformers[torch] datasets peft bitsandbytes tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 373.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 268.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 904.9 kB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip

In [1]:
import json

curated_ds = json.load(open("data_5000.json"))

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/pythia-160m"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["### End", "### Instruction:", "### Response:\n", "### Context:"]}
)

model = AutoModelForCausalLM.from_pretrained(model_id, revision="step130000")
# model = AutoModelForCausalLM.from_pretrained(model_id, revision="step143000")

In [12]:
def tokenize_alpaca_item(tokenizer, item):
    text = f'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: Give me a search term and nothing else for the following text: {item["original_instruction"]} {item["original_input"]} \n\n### Response:\n{item["search_term"]}\n\n### End### End### End'
    return tokenizer(text)

In [13]:
import tqdm

ds_tokenized = [tokenize_alpaca_item(tokenizer, item) for item in tqdm.tqdm(curated_ds["data"])]


100%|██████████| 5000/5000 [00:01<00:00, 3298.29it/s]


In [14]:
for tmp in tqdm.tqdm(ds_tokenized):
    # print(tmp['input_ids'][-1])
    # print(tokenizer("### End"))
    assert tmp['input_ids'][-1]==tokenizer("### End")['input_ids'][0]

100%|██████████| 5000/5000 [00:00<00:00, 23749.58it/s]


In [15]:
from peft import LoraConfig, get_peft_model, PeftModel

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM",
) 

peft_model = get_peft_model(model, config)
print_trainable_parameters(peft_model)

trainable params: 294912 || all params: 162617856 || trainable%: 0.18135277838123753


In [16]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=ds_tokenized,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        # max_steps=100,
        num_train_epochs=1,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=5,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
5,48.319900
10,44.002000
15,23.755800
20,12.751200
25,5.672200
30,3.913900
35,3.259800
40,2.930500
45,2.576200
50,2.406900


TrainOutput(global_step=156, training_loss=6.1069406637778645, metrics={'train_runtime': 201.917, 'train_samples_per_second': 24.763, 'train_steps_per_second': 0.773, 'total_flos': 361532512493568.0, 'train_loss': 6.1069406637778645, 'epoch': 0.9984})

In [29]:
item = curated_ds["data"][1001]
print(item)
PROMPT = f'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: Give me a search term and nothing else for the following text: {item["original_instruction"]} {item["original_input"]} \n\n### Response:\n'

answer = tokenizer.decode(peft_model.to('cuda').generate(**tokenizer(PROMPT, return_tensors="pt").to('cuda'), max_new_tokens=50)[0])
print(answer.split("### End")[0].split("### Response:\n")[1].split("\n")[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'command': 'Examine how individual and societal actions impact outcomes.', 'search_term': '"Effects of Individual vs Societal Performance"', 'explanation': 'The comparison between individual and societal performance in terms of achievement and productivity reveals that while individuals may excel in specific domains, societal achievements are often driven by the collective efforts and interdependence of group members.', 'instruction_prompt': '### Instruction: Examine how individual and societal actions impact outcomes. \n\n### Context: The comparison between individual and societal performance in terms of achievement and productivity reveals that while individuals may excel in specific domains, societal achievements are often driven by the collective efforts and interdependence of group members. \n\n### Response:\n', 'original_instruction': 'Compare and contrast the effects of individual and societal performance', 'original_input': '', 'original_output': 'Individual performance is rel

In [30]:
peft_model.save_pretrained("v1_pythia_160m_step130000-search_term")

In [31]:
tokenizer.save_pretrained("v1_pythia_160m_step130000-tokenizer_custom")

('v1_pythia_160m_step130000-tokenizer_custom/tokenizer_config.json',
 'v1_pythia_160m_step130000-tokenizer_custom/special_tokens_map.json',
 'v1_pythia_160m_step130000-tokenizer_custom/tokenizer.json')